<a href="https://colab.research.google.com/github/chelsea591/Econometria_QLab_PC/blob/main/PC1/PC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica Calificada 1**
*Integrantes:*
* Lesly Guzmán (20212335)
* Angie Quispe (20231371)
* Chelsea Torres (20200891)

# **Differences-in-Differences**

#**Ejercicio 1**
In this question, you will analyze the effect of China entering the WTO in 2001 on U.S. Manufacturing employment. Several papers attempt to tackle this issue (See Autor, Dorn, Hanson, 2013), we are going to mimic these papers using a simple framework. First, download from the Census website the County Business Patterns (CBP) series. This series includes the number of establishments, employment during the week of March 12, first quarter payroll, and annual payroll. CBP datasets are downloadable files in comma-separated value (CSV) format. In addition, corresponding record layouts and reference files are available. https://www.census.gov/programs-surveys/cbp/data/datasets.html


- See the attached do file which explains how to extract the dataset for year 1999 and keep all the 3-digit NAICS sector. You will have to do the same for years 1998-2006 and append all these files together. Create a variable called “year” that tracks the year of each dataset, this will be useful when appending all the years together. Keep the variables FIPSTATE, YEAR, NAICS, EMP, QP1, AP, and EST. Hint: Use the command “foreach var in “ ” “ to loop across years.

In [ ]:
# Instalamos e importamos librerías
%pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.9 MB/s eta 0:00:00


In [ ]:
# importamos librerías

import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Cargamos la base de datos
df = pd.read_stata('datacbp.dta')
df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38606 entries, 0 to 38605
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fipstate  38606 non-null  int8   
 1   naics     38606 non-null  object 
 2   emp       38606 non-null  int32  
 3   qp1       38606 non-null  int32  
 4   ap        38606 non-null  int32  
 5   est       38606 non-null  int32  
 6   year      38606 non-null  float32
dtypes: float32(1), int32(4), int8(1), object(1)
memory usage: 1.1+ MB


#### Descripción de las variables
* fipstate: código estatal
* naics: código de industria
* emp: total de empleados a mediados de marzo
* qp1: nómina total del primer trimestre
* ap: nómina anual total
* est: número total de establecimientos

In [ ]:
df['year']

,year
0,1998.0
1,1998.0
2,1998.0
3,1998.0
4,1998.0
...,...
38601,2006.0
38602,2006.0
38603,2006.0
38604,2006.0


In [ ]:
# Restringimos la muestra y concatenamos

processed_yearly_dfs = []

for year_val in range(1998, 2007):
    # Filtrar datos para el año actual
    df_year = df[df['year'] == year_val].copy()

    # Limpiar la columna "naics" para conservar solo los primeros tres caracteres numéricos
    df_year['naics'] = df_year['naics'].astype(str).str[:3]
    df_year = df_year[df_year['naics'].str.isnumeric()].copy()

    # Seleccionar columnas específicas
    selected_columns = ['fipstate', 'year', 'naics', 'emp', 'qp1', 'ap', 'est']
    df_year_processed = df_year[selected_columns]

    processed_yearly_dfs.append(df_year_processed)

# Concatenar todos los marcos de datos anuales procesados ​​en un único marco de datos
processed_df = pd.concat(processed_yearly_dfs, ignore_index=True)

processed_df.info()
display(processed_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38606 entries, 0 to 38605
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fipstate  38606 non-null  int8   
 1   year      38606 non-null  float32
 2   naics     38606 non-null  object 
 3   emp       38606 non-null  int32  
 4   qp1       38606 non-null  int32  
 5   ap        38606 non-null  int32  
 6   est       38606 non-null  int32  
dtypes: float32(1), int32(4), int8(1), object(1)
memory usage: 1.1+ MB


,fipstate,year,naics,emp,qp1,ap,est
0,1,1998.0,113,8207,44757,195720,1082
1,1,1998.0,114,93,363,1532,30
2,1,1998.0,115,1614,10181,45519,194
3,1,1998.0,211,804,9477,38146,38
4,1,1998.0,212,7074,79136,318295,159


### **1.  ¿Cuál es el nivel de observación?**

El nivel de observación de `processed_df` es a nivel de **fipstate-year-naics** (de 3 dígitos).

### **2. Construcción de la variable dummy `post_china`**



In [ ]:
processed_df['post_china'] = (processed_df['year'] >= 2001).astype(int)
processed_df

,fipstate,year,naics,emp,qp1,ap,est,post_china
0,1,1998.0,113,8207,44757,195720,1082,0
1,1,1998.0,114,93,363,1532,30,0
2,1,1998.0,115,1614,10181,45519,194,0
3,1,1998.0,211,804,9477,38146,38,0
4,1,1998.0,212,7074,79136,318295,159,0
...,...,...,...,...,...,...,...,...
38601,56,2006.0,721,7759,30345,202369,499,1
38602,56,2006.0,722,17741,46549,211741,1217,1
38603,56,2006.0,811,3274,23666,110002,721,1
38604,56,2006.0,812,1721,7451,31694,366,1


### **3. Construcción de la variable dummy `manuf`, donde `manuf = 1` para todas las observaciones que empiezan con el codigo "3" en naics (manufactura), de lo contrario `manuf = 0`**

In [ ]:
processed_df['manuf'] = (processed_df['naics'].astype(str).str.startswith('3')).astype(int)
processed_df

,fipstate,year,naics,emp,qp1,ap,est,post_china,manuf
0,1,1998.0,113,8207,44757,195720,1082,0,0
1,1,1998.0,114,93,363,1532,30,0,0
2,1,1998.0,115,1614,10181,45519,194,0,0
3,1,1998.0,211,804,9477,38146,38,0,0
4,1,1998.0,212,7074,79136,318295,159,0,0
...,...,...,...,...,...,...,...,...,...
38601,56,2006.0,721,7759,30345,202369,499,1,0
38602,56,2006.0,722,17741,46549,211741,1217,1,0
38603,56,2006.0,811,3274,23666,110002,721,1,0
38604,56,2006.0,812,1721,7451,31694,366,1,0


### **4. ¿Cuál es el efecto de la entrada de China en la OMC sobre el empleo?**
**Grupo de Tratamiento**: Industrias manufactureras (`manuf = 1`)

**Grupo de Control**: Industrias no manufactureras (`manuf = 0`)

**Período Post-intervención**: Año >= 2001 (`post_china = 1`)

**Período Pre-intervención**: Año < 2001 (`post_china = 0`)

In [ ]:
# Calculamos el empleo promedio para cada una de las cuatro celdas
did_means = processed_df.groupby(['manuf', 'post_china'])['emp'].mean().unstack()

print("Empleo promedio (emp) por grupo y período:")
display(did_means)

Empleo promedio (emp) por grupo y período:


post_china,0,1
manuf,,
0,29539.341747,30579.647311
1,15619.126095,13366.851284


In [ ]:
# Calculamos el efecto DiD
# (Empleo promedio post-tratamiento - Empleo promedio pre-tratamiento) - (Empleo promedio post-control - Empleo promedio pre-control)
did_effect = (did_means.loc[1, 1] - did_means.loc[1, 0]) - (did_means.loc[0, 1] - did_means.loc[0, 0])

print(f"El efecto de diferencias en diferencias (DiD) sobre el empleo es: {did_effect:.2f}")

El efecto de diferencias en diferencias (DiD) sobre el empleo es: -3292.58


* **Interpretación**: El empleo en las industrias manufactureras disminuyó *más* en el período post-tratamiento (entrada de China en la OMC) en comparación con las industrias no manufactureras. Es decir, la entrada de China en la OMC sobre el empleo en el sector manufacturero tuvo un impacrto negativo.

### **5. Regresión de DiD**

In [ ]:
import statsmodels.formula.api as smf

# Fórmula de la regresión DiD: emp = beta_0 + beta_1*manuf + beta_2*post_china + beta_3*(manuf * post_china)
formula_did = 'emp ~ manuf + post_china + manuf:post_china'

# Ajustar el modelo OLS
model_did = smf.ols(formula=formula_did, data=processed_df).fit()

print("\n--- RESULTADOS DE LA REGRESIÓN DiD ---")
print(model_did.summary())


--- RESULTADOS DE LA REGRESIÓN DiD ---
                            OLS Regression Results                            
Dep. Variable:                    emp   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     176.0
Date:                Thu, 05 Feb 2026   Prob (F-statistic):          2.44e-113
Time:                        06:44:40   Log-Likelihood:            -4.7943e+05
No. Observations:               38606   AIC:                         9.589e+05
Df Residuals:                   38602   BIC:                         9.589e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------


* **Interpretación:** Se puede observar que el coeficiente beta_3 o `manuf:post_china` del ítem 5 coincide con el resultado del ítem 4 que es -3292.5804

### **6. Regresión DiD para el número de establecimientos (`est`) y pago medio (`qp1/emp`).**

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

In [ ]:
# Creamos la variable de pago medio
avg_pay = processed_df['qp1'] / processed_df['emp']
processed_df['avg_pay'] = avg_pay

In [ ]:
# Primer modelo 'establecimiento' (est)
formula_est = 'est ~ manuf + post_china + manuf:post_china'

# Ajustar el modelo OLS para establecimientos
model_est = smf.ols(formula=formula_est, data=processed_df).fit()

print("\n--- RESULTADOS DiD: NÚMERO DE ESTABLECIMIENTOS (est) ---")
print(model_est.summary())


--- RESULTADOS DiD: NÚMERO DE ESTABLECIMIENTOS (est) ---
                            OLS Regression Results                            
Dep. Variable:                    est   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     425.9
Date:                Thu, 05 Feb 2026   Prob (F-statistic):          3.20e-272
Time:                        06:44:52   Log-Likelihood:            -3.7730e+05
No. Observations:               38606   AIC:                         7.546e+05
Df Residuals:                   38602   BIC:                         7.546e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

**Interpretación:** Se observa que el coeficiente de ```manuf:post_china``` (-52.57) tiene un P-value de 0.621. Al ser mayor a 0.05, no es estadísticamente significativo. Por lo tanto, no encontramos evidencia estadística para afirmar que la entrada de China a la OMC causó una disminución en el número de establecimientos en el sector manufacturero.



In [ ]:
# Segundo modelo 'Pago promedio' (avg_pay)
formula_pay = 'avg_pay ~ manuf + post_china + manuf:post_china'

# Ajustar el modelo OLS para establecimientos
model_pay = smf.ols(formula=formula_pay, data=processed_df).fit()

print("\n--- RESULTADOS DiD: NÚMERO DE PAGO PROMEDIO (avg_pay) ---")
print(model_pay.summary())


--- RESULTADOS DiD: NÚMERO DE PAGO PROMEDIO (avg_pay) ---
                            OLS Regression Results                            
Dep. Variable:                avg_pay   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     241.7
Date:                Wed, 04 Feb 2026   Prob (F-statistic):          3.76e-155
Time:                        23:57:10   Log-Likelihood:            -1.0015e+05
No. Observations:               33008   AIC:                         2.003e+05
Df Residuals:                   33004   BIC:                         2.003e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

* **Interpretación:** El coeficiente de la interacción es -0.2150 con un P-value de 0.111. Dado que el P-value excede el nivel de significancia estándar (0.10), el resultado no es estadísticamente significativo. Esto indica que no hay evidencia suficiente para concluir que la entrada de China a la OMC tuvo un impacto en los salarios promedio del sector manufacturero.

### **7. Regresión DiD con logartimo del empleo log(`emp`).**

In [ ]:
# Generamos la variable dependiente en logaritmos
log_emp = np.log(processed_df['emp'])
processed_df['log_emp'] = log_emp

processed_df['log_emp'] = np.log(processed_df['emp'] + 1)
# Fórmula de la regresión DiD: log_emp = beta_0 + beta_1*manuf + beta_2*post_china + beta_3*(manuf * post_china)
formula_logdid = 'log_emp ~ manuf + post_china + manuf:post_china'

# Ajustar el modelo OLS
model_logdid = smf.ols(formula=formula_logdid, data=processed_df).fit()

print("\n--- RESULTADOS DE LA REGRESIÓN DiD ---")
print(model_logdid.summary())


--- RESULTADOS DE LA REGRESIÓN DiD ---
                            OLS Regression Results                            
Dep. Variable:                log_emp   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     7.026
Date:                Thu, 05 Feb 2026   Prob (F-statistic):           0.000102
Time:                        06:45:32   Log-Likelihood:            -1.0403e+05
No. Observations:               38606   AIC:                         2.081e+05
Df Residuals:                   38602   BIC:                         2.081e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------


* **Interpretación:** Dado que el coeficiente de la variable ```manuf:post_china```  no es significativo (0.188), no se puede asegurar que la entrada de China a la Organización Mundial del Comercio tenga un efecto en la disminución del empleo manufaccturero.

* **¿Es necesario tomar logaritmos?**

  Tomar logaritmos es recomendable en este contexto por dos razones principales. Primero, las variables de empleo suelen tener una distribución sesgada a la derecha, y el logaritmo suaviza estos valores atípicos acercando la distribución a la normalidad. Segundo, permite interpretar el coeficiente como una elasticidad o cambio porcentual, lo cual ermite interpretar los efectos en términos relativos (%) en lugar de absolutos.


## **Event Study**

## **8. Generación de dummies por año. y la interacción entre estas y el grupo de tratamiento (`manuf`).**

In [ ]:
# Generamos dummies por año
processed_df['year'] = processed_df['year'].astype(int)

year_dummies = pd.get_dummies(processed_df['year'], prefix='year')
processed_df = pd.concat([processed_df, year_dummies], axis=1)

In [ ]:
# Generamos las interacciones
unique_years = sorted(processed_df['year'].unique())
for y in unique_years:
    interaction_name = f'interaction_{y}'
    processed_df[f'interaction_{y}'] = processed_df['manuf'] * processed_df[f'year_{y}']

print(f"Años encontrados: {unique_years}")
print("Columnas de interacción creadas:")
print([col for col in processed_df.columns if 'interaction_' in col])

Años encontrados: [np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006)]
Columnas de interacción creadas (deberían ser 9):
['interaction_1998', 'interaction_1999', 'interaction_2000', 'interaction_2001', 'interaction_2002', 'interaction_2003', 'interaction_2004', 'interaction_2005', 'interaction_2006']


## **9. Regresión con log(`manuf`) vs. las dummies por año (Efectos fijos)**

In [ ]:
# Definimos las interacciones
interaction_vars = [f'interaction_{y}' for y in range(1999, 2007)]
interactions_formula = " + ".join(interaction_vars)

In [ ]:
# Construimos las fórmula
# log_emp ~ Interacciones + Efectos Fijos de Año + Efectos Fijos de Estado + Efectos Fijos NAICS
formula_es = f"log_emp ~ {interactions_formula} + C(year) + C(fipstate) + C(naics)"
model_es = smf.ols(formula=formula_es, data=processed_df).fit()

print("\n--- RESULTADOS EVENT STUDY: LOG(EMPLEO) ---")
print(model_es.summary().tables[1])


--- RESULTADOS EVENT STUDY: LOG(EMPLEO) ---
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             3.1278      0.123     25.530      0.000       2.888       3.368
C(year)[T.1999]      -0.1820      0.050     -3.631      0.000      -0.280      -0.084
C(year)[T.2000]      -0.1135      0.050     -2.264      0.024      -0.212      -0.015
C(year)[T.2001]      -0.0687      0.050     -1.371      0.171      -0.167       0.030
C(year)[T.2002]       0.1358      0.050      2.710      0.007       0.038       0.234
C(year)[T.2003]       0.1370      0.050      2.727      0.006       0.039       0.236
C(year)[T.2004]       0.1036      0.050      2.061      0.039       0.005       0.202
C(year)[T.2005]       0.1780      0.050      3.542      0.000       0.079       0.277
C(year)[T.2006]       0.1105      0.050      2.198      0.028       0.012       0.209
C(fipstat

* **Should you expect to see any effect for the interaction term ```manuf * year``` 1999 or ```manuf * year``` 2000?**

  No se esperaría un efecto de la interacción ```manuf * year``` en ninguno de los años mencionados debido a que el shock (la inclusión de China en la OMC) se dió a partir del año 2001. Ambos años son tomados para el pre-tratamiento.

  Asimismo, se esperaría que los coeficientes de estas dos interacciones sean insignificativas para confirmar el supuesto de **tendencias paralelas**. Sin embargo, se observa que en ```interaction_2000``` el coeficiente **es significativo estadísticamente** al 10%, lo que implica que podría haber tendencias previas al shock. Esto sugiere que el sector manufacturero ya podría haber estado experimentando cambios diferenciales justo antes de la entrada de China.

* **Did the China shock have a significant effect on employment? Was it a short-run or long-run effect?**

  Se observa que los coeficientes de las interacciones son **estadísticamente significativos** en los años 2002 (al 1%), 2004 (al 10%), 2005 (al 5%) y 2006 (al 10%). Esto significa que para estos años, el empleo en el sector manufacturero se vio afectado negativamente debido a la integración de China a la Organización Mundial del Comercio.

  Por otro lado, se observa que los efectos negativos tienen un retraso de un año (lag), ya que no son inmediatos en 2001, sino que empiezan en el año 2002. Estos efectos son de largo plazo, prolongándose hasta el año 2006, aunque se observa una ausencia de significancia estadística puntual en el año 2003. En general, la evidencia apunta a un impacto negativo persistente en el tiempo.

## **10.Regresión con el logartimo de establecimientos (log(`est`)) y pago medio (`avg_pay`)**

In [ ]:
# Generamos log(est)
log_est = np.log(processed_df['est'])
processed_df['log_est'] = log_est

In [ ]:
# Definimos las interacciones
interaction_vars= [f'interaction_{y}' for y in range(1999, 2007)]
interactions_formula = " + ".join(interaction_vars)

In [ ]:
# Establecimientos: log(est)
formula_est_es = f"log_est ~ {interactions_formula} + C(year) + C(fipstate) + C(naics)"
model_est_es = smf.ols(formula=formula_est_es, data=processed_df).fit()

print("\n--- EVENT STUDY: LOG(ESTABLECIMIENTOS) ---")
print(model_est_es.summary().tables[1])


--- EVENT STUDY: LOG(ESTABLECIMIENTOS) ---
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             4.6645      0.035    131.446      0.000       4.595       4.734
C(year)[T.1999]       0.0122      0.015      0.838      0.402      -0.016       0.041
C(year)[T.2000]       0.0291      0.015      2.007      0.045       0.001       0.058
C(year)[T.2001]       0.0464      0.015      3.195      0.001       0.018       0.075
C(year)[T.2002]       0.0691      0.015      4.759      0.000       0.041       0.098
C(year)[T.2003]       0.0798      0.015      5.485      0.000       0.051       0.108
C(year)[T.2004]       0.0885      0.015      6.078      0.000       0.060       0.117
C(year)[T.2005]       0.1007      0.015      6.914      0.000       0.072       0.129
C(year)[T.2006]       0.1097      0.015      7.532      0.000       0.081       0.138
C(fipstate

* **Interpretación:** En este caso, se observa que ```interaction_1999``` y ```interaction_2000``` tienen coeficientes no significativos estadísticamente, lo que indica que se cumple el supuesto de **tendencias paralelas**.

  Por otro lado, el shock de la integración de China a la OMC tiene un efecto negativo persistente a largo plazo en las fabricas. Este shock ocurre con un año de retraso como en el caso del empleo.


In [ ]:
# Pago medio: avg_pay
formula_avg = f"avg_pay ~ {interactions_formula} + C(year) + C(fipstate) + C(naics)"
model_avg = smf.ols(formula=formula_avg, data=processed_df).fit()

print("\n--- EVENT STUDY: Salario medio ---")
print(model_avg.summary().tables[1])


--- EVENT STUDY: LOG(ESTABLECIMIENTOS) ---
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             4.6128      0.255     18.103      0.000       4.113       5.112
C(year)[T.1999]       0.2858      0.077      3.726      0.000       0.135       0.436
C(year)[T.2000]       0.9115      0.076     11.920      0.000       0.762       1.061
C(year)[T.2001]       1.2575      0.076     16.448      0.000       1.108       1.407
C(year)[T.2002]       1.2760      0.076     16.845      0.000       1.128       1.424
C(year)[T.2003]       1.3475      0.076     17.641      0.000       1.198       1.497
C(year)[T.2004]       1.6183      0.076     21.158      0.000       1.468       1.768
C(year)[T.2005]       1.9119      0.076     25.024      0.000       1.762       2.062
C(year)[T.2006]       2.6456      0.077     34.552      0.000       2.495       2.796
C(fipstate

* **Interpretación:** En caso del pago promedio (```avg_pay```), en los años pre-tratamiento (1999 y 2000) su coeficientes son no significativos estadísticamente, lo que indica que se cumple el supuesto de **tendencias paralelas**.

  Asimismo, a diferencia de los casos anteriores, la integración de China a la OMC para los años evaluados (2001-2006) no es significativo en el pago medio. Los p-value de los coeficientes de las interacciones de estos años son mayores al nivel de significancia estándar (0.10).

#**Ejercicio 2**

This exercise is based on the paper, Eissa, Nada, and Jeffrey B. Liebman. 1996. Labor Supply Responses to the Earned Income Tax Credit (EITC). Quarterly Journal of Economics. 111(2): 605-637. The data “eitc.dta” file contains data for single women 20-54 with less than high school education, as this group is most likely to be affected by the EITC.  

In [ ]:
# Cargamos la base de datos
df = pd.read_stata('C:/Users/LESLY/Downloads/eitc.dta')

## **1: Tabla resumen de todos los datos**

In [ ]:
print("--- RESUMEN DE DATOS ---")
# describe() nos da media, desviación estándar, min, max, etc.
summary_table = df.describe()
display(summary_table)

--- RESUMEN DE DATOS ---


,state,year,urate,children,nonwhite,finc,earn,age,ed,work,unearn
count,13746.00000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000,13746.000000
mean,54.52459,1993.347046,6.761734,1.192638,0.600684,15255.319310,10432.475514,35.209661,8.806053,0.513022,4.822844
std,27.13521,1.709800,1.462461,1.382105,0.489776,19444.249684,18200.758138,10.157128,2.635639,0.499849,7.122624
min,11.00000,1991.000000,2.600000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000
25%,31.00000,1992.000000,5.700000,0.000000,0.000000,5123.417782,0.000000,26.000000,7.000000,0.000000,0.000000
50%,56.00000,1993.000000,6.800000,1.000000,1.000000,9636.664360,3332.179931,34.000000,10.000000,1.000000,2.973197
75%,81.00000,1995.000000,7.700000,2.000000,1.000000,18659.178178,14321.223709,44.000000,11.000000,1.000000,6.863863
max,95.00000,1996.000000,11.400000,9.000000,1.000000,575616.820870,537880.611850,54.000000,11.000000,1.000000,134.057520


## **2: Medias muestrales por grupo de hijos**

In [ ]:
print("\n--- MEDIAS POR GRUPO DE HIJOS ---")

# Creamos una función para clasificar a las mujeres según los grupos:
# (a) Sin hijos, (b) 1 hijo, (c) 2 o más hijos

def clasificar_hijos(n):
    if n == 0:
        return 'a) Sin hijos'
    elif n == 1:
        return 'b) 1 hijo'
    else:
        return 'c) 2+ hijos'

# Aplicamos la clasificación
df['grupo_hijos'] = df['children'].apply(clasificar_hijos)

# Calculamos el promedio de TODAS las variables agrupando por esta nueva categoría
medias_por_grupo = df.groupby('grupo_hijos').mean(numeric_only=True).T

display(medias_por_grupo)


--- MEDIAS POR GRUPO DE HIJOS ---


grupo_hijos,a) Sin hijos,b) 1 hijo,c) 2+ hijos
state,53.396660,55.590908,55.243855
year,1993.365479,1993.338135,1993.329956
urate,6.663067,6.802060,6.858664
children,0.000000,1.000000,2.801092
nonwhite,0.515944,0.596468,0.708885
finc,18559.863110,13941.569547,11985.295298
earn,13760.255844,9928.278759,6613.546648
age,38.498228,33.758993,32.047469
ed,8.548676,8.992479,9.006721
work,0.574490,0.537606,0.420710


## **3: Construir variables 'anykids' y 'post93'**

In [ ]:
# Variable de tratamiento (anykids): 1 si tiene al menos un hijo, 0 si no
df['anykids'] = (df['children'] >= 1).astype(int)

# Variable temporal (post93): 1 si el año es 1994 o posterior, 0 si no
df['post93'] = (df['year'] >= 1994).astype(int)


In [ ]:
# Creamos el término de interacción (el DiD)
df['interaction'] = df['anykids'] * df['post93']

# Verificamos que se hayan creado bien
print(df[['year', 'children', 'anykids', 'post93', 'interaction']].head())

     year  children  anykids  post93  interaction
0  1991.0         0        0       0            0
1  1991.0         1        1       0            0
2  1991.0         2        1       0            0
3  1991.0         0        0       0            0
4  1991.0         3        1       0            0


## **4: Regresión Diferencias en Diferencias**

In [ ]:
# TNuestro modelo sería: earn = beta0 + beta1*anykids + beta2*post93 + beta3*interacción
# El coeficiente de interés es 'interaction' (beta3)

model_basic = smf.ols(formula='earn ~ anykids + post93 + interaction', data=df).fit()

print("\n--- RESULTADOS DE LA REGRESIÓN ---")
print(model_basic.summary())


--- RESULTADOS DE LA REGRESIÓN ---
                            OLS Regression Results                            
Dep. Variable:                   earn   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     124.3
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           1.96e-79
Time:                        11:17:13   Log-Likelihood:            -1.5416e+05
No. Observations:               13746   AIC:                         3.083e+05
Df Residuals:                   13742   BIC:                         3.084e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     

- Interpretación: Viendo el coeficiente de la variable "interaction" (con un p-valor 0.007, que es estadisticamente significativo) podemos decir que, tras la expansión del programa EITC, las ganancias (earnings) de las mujeres solteras con hijos aumentaron en promedio $1669 dólares más que las de las mujeres solteras sin hijos.

## **5: Modelo con Efectos Fijos de Estado y Año**

Ahora debemos repetir el modelo (iv) pero con FE de estado y año.

In [ ]:
# Fórmula: earn = anykids + interaction + Efectos Fijos
# C(state) crea dummies por estado, C(year) crea dummies por año.
model_fe = smf.ols(formula='earn ~ anykids + interaction + C(state) + C(year)', data=df).fit()

print("--- RESULTADOS 5 (Efectos Fijos) ---")
# Buscamos y mostramos las filas de 'anykids' e 'interaction'
results_as_html = model_fe.summary().tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
display(result_df.loc[['anykids', 'interaction']])

--- RESULTADOS (Efectos Fijos) ---


,coef,std err,t,P>|t|,[0.025,0.975]
anykids,-6646.0405,420.540,-15.804,0.000,-7470.357,-5821.724
interaction,1612.1312,617.393,2.611,0.009,401.956,2822.307


- El nuevo coeficiente de interaccion es similar al resultado de la Pregunta 4 (que era casi 1669). Esto sugiere que el efecto estimado es robusto; es decir, no cambia mucho incluso cuando controlamos por las diferencias fijas entre estados y los choques anuales comunes.

## **6: Modelo con Controles Adicionales**

In [ ]:
# Agregamos: urate (desempleo), nonwhite, age, ed (educación), unearn (ingreso no laboral)

model_controls = smf.ols(
    formula='earn ~ anykids + interaction + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df
).fit()

print("\n--- RESULTADOS 6 (Con Controles) ---")
results_as_html_cont = model_controls.summary().tables[1].as_html()
result_df_cont = pd.read_html(results_as_html_cont, header=0, index_col=0)[0]
display(result_df_cont.loc[['anykids', 'interaction', 'urate', 'nonwhite', 'age', 'ed', 'unearn']])


--- RESULTADOS 6 (Con Controles) ---


,coef,std err,t,P>|t|,[0.025,0.975]
anykids,-6224.6707,431.618,-14.422,0.000,-7070.702,-5378.640
interaction,1661.8117,616.237,2.697,0.007,453.903,2869.720
urate,-44.8069,276.405,-0.162,0.871,-586.599,496.985
nonwhite,-2575.0164,361.745,-7.118,0.000,-3284.087,-1865.946
age,33.5963,15.934,2.108,0.035,2.363,64.830
ed,119.9487,60.318,1.989,0.047,1.717,238.180
unearn,-42.7092,21.670,-1.971,0.049,-85.186,-0.233


- Nuevamente, el resultado del coeficiente es muy estable (similar a 1668 y 1612). Esto refuerza la conclusión de que el programa EITC tuvo un efecto positivo y significativo en las ganancias, independientemente de la edad, educación o ubicación de las mujeres.

## **7: Efectos Variados (1 hijo vs 2+ hijos)**

In [ ]:
# 1. Creamos dummies para los grupos específicos de hijos
df['one_kid'] = (df['children'] == 1).astype(int)
df['two_plus_kids'] = (df['children'] >= 2).astype(int)

# 2. Creamos las interacciones con post93
df['interact_1'] = df['one_kid'] * df['post93']
df['interact_2'] = df['two_plus_kids'] * df['post93']

# 3. Corremos la regresión incluyendo los controles de la pregunta 6
# Nota: Usamos 'one_kid' y 'two_plus_kids' en lugar de 'anykids' para separar los niveles base también.

model_hetero = smf.ols(
    formula='earn ~ one_kid + two_plus_kids + interact_1 + interact_2 + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df
).fit()

print("--- RESULTADOS 7 (Efectos por N° de Hijos) ---")
# Mostramos las variables de interés
results_as_html_h = model_hetero.summary().tables[1].as_html()
result_df_h = pd.read_html(results_as_html_h, header=0, index_col=0)[0]
display(result_df_h.loc[['interact_1', 'interact_2']])

--- RESULTADOS 7 (Efectos por N° de Hijos) ---


,coef,std err,t,P>|t|,[0.025,0.975]
interact_1,2699.6643,795.516,3.394,0.001,1140.344,4258.984
interact_2,953.5636,694.884,1.372,0.170,-408.505,2315.632


- Contrario a la predicción basada en el monto del subsidio (que es mayor para familias grandes), los resultados muestran que la política fue efectiva solo para madres con un hijo (por el coeficiente de 2699.6, que tiene un valor p (0.001) significativo).
  
- En el mundo real, esto puede deberse a que los costos fijos y variables de trabajar (especialmente cuidado infantil y logística) crecen exponencialmente con el número de hijos. Es posible que el subsidio adicional por el segundo hijo no sea suficiente para compensar el costo de mercado de su cuidado, haciendo que la barrera para entrar al mercado laboral siga siendo demasiado alta para madres con familias numerosas.

## **8: Prueba Placebo (Datos pre-reforma)**

In [ ]:
# 1. Filtramos datos: Usar solo años ANTERIORES a la reforma real (<= 1993)
df_placebo = df[df['year'] <= 1993].copy()

# 2. Creamos una reforma "falsa" en 1992
# Asumimos que la política empezó en 1992 (tratamiento: 92 y 93)
df_placebo['post92_fake'] = (df_placebo['year'] >= 1992).astype(int)

# 3. Creamos interacción placebo
df_placebo['interaction_placebo'] = df_placebo['anykids'] * df_placebo['post92_fake']

# 4. Corremos la regresión con controles
model_placebo = smf.ols(
    formula='earn ~ anykids + interaction_placebo + urate + nonwhite + age + ed + unearn + C(state) + C(year)',
    data=df_placebo
).fit()

print("\n--- RESULTADOS 8 (Prueba Placebo) ---")
results_as_html_p = model_placebo.summary().tables[1].as_html()
result_df_p = pd.read_html(results_as_html_p, header=0, index_col=0)[0]
display(result_df_p.loc[['interaction_placebo']])


--- RESULTADOS 8 (Prueba Placebo) ---


,coef,std err,t,P>|t|,[0.025,0.975]
interaction_placebo,942.4213,769.433,1.225,0.221,-565.888,2450.73


- Observamos que el coeficiente de interaction_placebo NO es significativo (por su valor-p de 0.2, que es mayor que 0.05).
- Ya que el coeficiente no es significativo, los resultados son tranquilizadores, ya que esto significa que antes de la reforma real, no había tendencias extrañas haciendo que las mujeres con hijos ganaran más.
- Esto valida que el efecto que encontramos en las preguntas anteriores fue causado realmente por la reforma de 1994 y no por otra cosa.